In [162]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import seaborn as sns
import datetime, tqdm
import os
import matplotlib.patches as patches
pd.set_option('max_columns', 100)
#from kaggle.competitions import nflrush
from sklearn.model_selection import KFold, RepeatedKFold
import math
import scipy
import numpy as np
import pandas as pd
import sklearn.metrics as mtr
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.callbacks import Callback, EarlyStopping
import gc
import os
#note： 
#1. As a result it might not be worthwhile to use features related to game clock/quarter of the game。
#2. There is no relationships between number of rushes before and running yards gained。
#3. rushing success larger depends on defender in box, or defender that are close to offensive lineman and attempt 
#   to counter the blocking.
#4. highly drafted player has the same average rushing yards as the rest
#ARI: Arizona Cardinals
#ATL: Atlanta Falcons
#BAL: Baltimore Ravens
#BUF: Buffalo Bills
#CAR: Carolina Panthers
#CHI: Chicago Bears
#CIN: Cincinnati Bengals
#CLE: Cleveland Browns
#DAL: Dallas Cowboys
#DEN: Denver Broncos
#DET: Detroit Lions
#GB: Green Bay Packers
#HOU: Houston Texans
#IND: Indianapolis Colts
#JAX: Jacksonville Jaguars
#KC: Kansas City Chiefs
#MIA: Miami Dolphins
#MIN: Minnesota Vikings
#NE: New England Patriots
#NO: New Orleans Saints
#NYG: New York Giants
#NYJ: New York Jets
#OAK: Oakland Raiders
#PHI: Philadelphia Eagles
#PIT: Pittsburgh Steelers
#SD: San Diego Chargers
#SEA: Seattle Seahawks
#SF: San Francisco 49ers
#STL: Saint Louis Rams
#TB: Tampa Bay Buccaneers
#TEN Tennessee Titans
#WAS: Washington Redskins

In [1]:
!pip install --upgrade keras

Requirement not upgraded as not directly required: h5py in c:\users\yam\appdata\local\continuum\anaconda3\lib\site-packages (from keras) (2.7.1)
Requirement not upgraded as not directly required: keras-preprocessing>=1.0.5 in c:\users\yam\appdata\local\continuum\anaconda3\lib\site-packages (from keras) (1.1.0)
Requirement not upgraded as not directly required: pyyaml in c:\users\yam\appdata\local\continuum\anaconda3\lib\site-packages (from keras) (3.12)
Requirement not upgraded as not directly required: scipy>=0.14 in c:\users\yam\appdata\local\continuum\anaconda3\lib\site-packages (from keras) (1.1.0)
Requirement not upgraded as not directly required: six>=1.9.0 in c:\users\yam\appdata\local\continuum\anaconda3\lib\site-packages (from keras) (1.11.0)
Requirement not upgraded as not directly required: numpy>=1.9.1 in c:\users\yam\appdata\local\continuum\anaconda3\lib\site-packages (from keras) (1.14.3)
Requirement not upgraded as not directly required: keras-applications>=1.0.6 in c:\u

tabpy-server 0.2 requires futures, which is not installed.
msgpack-numpy 0.4.4.2 requires msgpack>=0.5.2, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.
Cache entry deserialization failed, entry ignored
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [102]:
train_df = pd.read_csv(r'\\Neptune\RiskMgmtArchive\3 Temp Hold\Temp_Jason\research\train.csv')

C:\Users\yam\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [165]:
#team abbreviations correct
def data_clean(df):
#correct name   
    df.loc[df['PossessionTeam'] == 'ARZ', 'PossessionTeam'] = 'ARI'
    df.loc[df['PossessionTeam'] == 'BLT', 'PossessionTeam'] = 'BAL'
    df.loc[df['PossessionTeam'] == 'CLV', 'PossessionTeam'] = 'CLE'
    df.loc[df['PossessionTeam'] == 'HST', 'PossessionTeam'] = 'HOU'
    df.loc[df['FieldPosition'] == 'ARZ', 'FieldPosition'] = 'ARI'
    df.loc[df['FieldPosition'] == 'BLT', 'FieldPosition'] = 'BAL'
    df.loc[df['FieldPosition'] == 'CLV', 'FieldPosition'] = 'CLE'
    df.loc[df['FieldPosition'] == 'HST', 'FieldPosition'] = 'HOU'

# fill null
    df = df.fillna(df.median())

# offense time and defence time
    df['TeamOnOffense'] = "home"
    df.loc[df.PossessionTeam != df.HomeTeamAbbr, 'TeamOnOffense'] = "away"
    df['IsOnOffense'] = df.Team == df.TeamOnOffense # Is player on offense?
    
#time
    df['TimeHandoff'] = df['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['TimeSnap'] = df['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['PlayerBirthDate'] = df['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))
    seconds_in_year = 60*60*24*365.25
    df['PlayerAge'] = df.apply(lambda row: (row['TimeHandoff']-row['PlayerBirthDate']).total_seconds()/seconds_in_year, axis=1)
    df['GameClock'] = df['GameClock'].apply(lambda x: float(x.split(":")[0]) + float(x.split(":")[1])/60)
    df['TimeDelta'] = df.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)
#player height
    df['PlayerHeight'] = df['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))

#weather
    def map_weather(txt):
        ans = 1
        if pd.isna(txt):
            return 0
        if 'partly' in txt:
            ans*=0.5
        if 'climate controlled' in txt or 'indoor' in txt:
            return ans*3
        if 'sunny' in txt or 'sun' in txt:
            return ans*2
        if 'clear' in txt:
            return ans
        if 'cloudy' in txt:
            return -ans
        if 'rain' in txt or 'rainy' in txt:
            return -2*ans
        if 'snow' in txt:
            return -3*ans
        return 0
    df['Cleaned_GameWeather'] = df['GameWeather'].str.lower()
    indoor = "indoor"
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: indoor if not pd.isna(x) and indoor in x else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['GameWeather'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
    df['Cleaned_GameWeather'] = df['Cleaned_GameWeather'].apply(map_weather)

#diff Score    
    df["DiffScoreBeforePlay_ob"] = (df["HomeScoreBeforePlay"] - df["VisitorScoreBeforePlay"])
    df.loc[df['Team'] == 'away',["DiffScoreBeforePlay_ob"]] = - df.loc[df['Team'] == 'away',["DiffScoreBeforePlay_ob"]]
#Turf
    Turf = {'Field Turf':'Artificial', 'A-Turf Titan':'Artificial', 'Grass':'Natural', 'UBU Sports Speed S5-M':'Artificial', 'Artificial':'Artificial', 'DD GrassMaster':'Artificial', 'Natural Grass':'Natural', 'UBU Speed Series-S5-M':'Artificial', 'FieldTurf':'Artificial', 'FieldTurf 360':'Artificial', 'Natural grass':'Natural', 'grass':'Natural', 'Natural':'Natural', 'Artifical':'Artificial', 'FieldTurf360':'Artificial', 'Naturall Grass':'Natural', 'Field turf':'Artificial', 'SISGrass':'Artificial', 'Twenty-Four/Seven Turf':'Artificial', 'natural grass':'Natural'} 
    df['Cleaned_Turf'] = df['Turf'].map(Turf)

#left to right
    df['New_X'] = df['X']
    df.loc[df['PlayDirection'] == 'left','New_X'] = 120 - df.loc[df['PlayDirection'] == 'left','X']
    df['New_Y'] = df['Y']
    df.loc[df['PlayDirection'] == 'left','New_Y'] = 160/3 - df.loc[df['PlayDirection'] == 'left','Y']
    df['Orientation_std'] = df['Orientation']
    df.loc[df['Season'] == 2017, 'Orientation_std'] = df.loc[df['Season'] == 2017, 'Orientation_std'] + 90
    df.loc[df['PlayDirection'] == 'left', 'Orientation_std'] = np.mod(180 + df.loc[df['PlayDirection'] == 'left', 'Orientation_std'], 360)
    df['Dir_std'] = df['Dir']
    df.loc[df['PlayDirection'] == 'left', 'Dir_std'] = np.mod(180 + df.loc[df['PlayDirection'] == 'left', 'Dir_std'], 360)
    df['YardLine_std'] = 100 - df['YardLine']
    df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
          'YardLine_std'
         ] = df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
          'YardLine']
    df["Orientation_sin"] = df["Orientation_std"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    df["Orientation_cos"] = df["Orientation_std"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
    df["Dir_sin"] = df["Dir_std"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
    df["Dir_cos"] = df["Dir_std"].apply(lambda x : np.cos(x/360 * 2 * np.pi))

#distance
    #distance to yardline
    df['Dis_YardLine'] = df['New_X'] - df['YardLine_std'] - 10
    #distance to rusher
    def Distance(x1,x2,y1,y2):
        x_diff = (x1-x2)**2
        y_diff = (y1-y2)**2
        return np.sqrt(x_diff + y_diff)
    def Degree(x1,x2,y1,y2):
        try:
            tan = (y1-y2)/(x1-x2)
        except:
            tan = 0
        degree = 90 - math.atan(tan)/(2*np.pi)*360
        return degree
    df['IsRusher'] = (df['NflId'] == df['NflIdRusher'])
    Rusher =df.loc[df['IsRusher'],['PlayId','X','Y','Dir_std']].rename(columns={"X":"Rusher_X","Y":"Rusher_Y",'PossessionTeam':'Offense_Team','Dir_std':'Rusher_Dir_std'})
    df = df.merge(Rusher,how = 'left',on = 'PlayId')
    df['Distance_to_Rusher'] = df[["X","Rusher_X","Y","Rusher_Y"]].apply(lambda x: Distance(x[0],x[1],x[2],x[3]), axis = 1)
    df['Degree_to_Rusher'] = df[["X","Rusher_X","Y","Rusher_Y"]].apply(lambda x: Degree(x[0],x[1],x[2],x[3]), axis = 1)
    df['Degree_Diff'] = abs(df['Degree_to_Rusher'] - df['Rusher_Dir_std'])
    #back_direction
    def back_direction(orientation):
        if orientation > 180.0:
            return 1
        else:
            return 0
    df['back_oriented_down_field'] = df['Orientation_std'].apply(lambda x: back_direction(x))

# Stadium clean
    map_stad = {'Broncos Stadium at Mile High': 'Broncos Stadium At Mile High', 'CenturyField': 'CenturyLink Field', 'CenturyLink': 'CenturyLink Field', 'Everbank Field': 'EverBank Field', 'FirstEnergy': 'First Energy Stadium', 'FirstEnergy Stadium': 'First Energy Stadium', 'FirstEnergyStadium': 'First Energy Stadium', 'Lambeau field': 'Lambeau Field', 'Los Angeles Memorial Coliesum': 'Los Angeles Memorial Coliseum', 'M & T Bank Stadium': 'M&T Bank Stadium', 'M&T Stadium': 'M&T Bank Stadium', 'Mercedes-Benz Dome': 'Mercedes-Benz Superdome', 'MetLife': 'MetLife Stadium', 'Metlife Stadium': 'MetLife Stadium', 'NRG': 'NRG Stadium', 'Oakland Alameda-County Coliseum': 'Oakland-Alameda County Coliseum', 'Paul Brown Stdium': 'Paul Brown Stadium', 'Twickenham': 'Twickenham Stadium'}
    
    for stad in df['Stadium'].unique():
        if stad in map_stad.keys():
            pass
        else:
            map_stad[stad]=stad

    df['Stadium'] = df['Stadium'].map(map_stad)

# wind 
    def give_me_WindSpeed(x):
            x = str(x)
            x = x.replace('mph', '').strip()
            if '-' in x:
                x = (int(x.split('-')[0]) + int(x.split('-')[1])) / 2
            elif 'gusts up to' in x:
                x = (int(x.split()[0]) + int(x.split()[-1])) / 2
            elif 'clam' in x:
                x = 0
            try:
                return float(x)
            except:
                return -99
    df['Cleaned_WindSpeed'] = df['WindSpeed'].apply(give_me_WindSpeed)

    
# speed
    df['Horizontal Speed'] = df['S']*df["Dir_sin"]
    df['Vertical Speed'] = df['S']*df["Dir_cos"]

    
    return df

In [166]:
train = data_clean(train_df)

C:\Users\yam\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:97: RuntimeWarning: invalid value encountered in float_scalars
C:\Users\yam\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in float_scalars


In [167]:
def split_data(df):
    df1 = df.loc[df['IsRusher']]
    df2 = df.loc[df['IsOnOffense'] & (~df['IsRusher'])]
    df3 = df.loc[~df['IsOnOffense']]

# handoff times
    df1 = df1.sort_values(['GameId','PlayId','Quarter','GameClock'])
    df1['# of Handoff Play'] = 1
    df1['# of Handoff Play'] = df1[['GameId','PlayId','# of Handoff Play']].groupby(['GameId','PlayId']).cumsum()
# min_time_to_tacke
    df3['Min_Time_Tacke'] = df3['Distance_to_Rusher']/df3['S']

# defence_X_Y_spread
    Defence_X_Y_std = df3[["PlayId",'New_X','New_Y']].groupby("PlayId").std().rename(columns={'New_X':'Defense_X_std','New_Y':'Defense_Y_std'}) \
    .reset_index()
    
    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_X_Removed2_std = df3[["PlayId",'New_X']].drop(np.hstack([df3.groupby('PlayId').tail(1).index, df3.groupby('PlayId').head(1).index]))\
    .groupby('PlayId').std().rename(columns={'New_X':'Defense_X_Removed2_std'}).reset_index()

    df3 = df3.sort_values(['PlayId','New_Y'])
    Defense_Y_Removed2_std = df3[["PlayId",'New_Y']].drop(np.hstack([df3.groupby('PlayId').tail(1).index, df3.groupby('PlayId').head(1).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Defense_Y_Removed2_std'}).reset_index()
    
    df3 = df3.sort_values(['PlayId','New_X'])
    Defense_X_Removed4_std = df3[["PlayId",'New_X']].drop(np.hstack([df3.groupby('PlayId').tail(2).index, df3.groupby('PlayId').head(2).index]))\
    .groupby('PlayId').std().rename(columns={'New_X':'Defense_X_Removed4_std'}).reset_index()
    
    df3 = df3.sort_values(['PlayId','New_Y'])
    Defense_Y_Removed4_std = df3[["PlayId",'New_Y']].drop(np.hstack([df3.groupby('PlayId').tail(2).index, df3.groupby('PlayId').head(2).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Defense_Y_Removed4_std'}).reset_index()
    df1 = df1.merge(Defence_X_Y_std, how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_X_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_Y_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_X_Removed4_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Defense_Y_Removed4_std,how = 'left',  on ='PlayId')

# offense_X_Y_spread
    df2 = df2.sort_values(['PlayId','New_X'])
    Offense_X_Removed2_std = df2[["PlayId",'New_X']].drop(np.hstack([df2.groupby('PlayId').tail(1).index, df2.groupby('PlayId').head(1).index])) \
    .groupby('PlayId').std().rename(columns={'New_X':'Offense_X_Removed2_std'}).reset_index()
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_Y_Removed2_std = df2[["PlayId",'New_Y']].drop(np.hstack([df2.groupby('PlayId').tail(1).index, df2.groupby('PlayId').head(1).index]))\
    .groupby('PlayId').std().rename(columns={'New_Y':'Offense_Y_Removed2_std'}).reset_index()
    
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_X_Removed4_std = df2[["PlayId",'New_X']].drop(np.hstack([df2.groupby('PlayId').tail(2).index, df2.groupby('PlayId').head(2).index])) \
    .groupby('PlayId').std().rename(columns={'New_X':'Offense_X_Removed4_std'}).reset_index()
    df2 = df2.sort_values(['PlayId','New_Y'])
    Offense_Y_Removed4_std = df2[["PlayId",'New_Y']].drop(np.hstack([df2.groupby('PlayId').tail(2).index, df2.groupby('PlayId').head(2).index])) \
    .groupby('PlayId').std().rename(columns={'New_Y':'Offense_Y_Removed4_std'}).reset_index()
    df1 = df1.merge(Offense_X_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_Y_Removed2_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_X_Removed4_std,how = 'left',  on ='PlayId')
    df1 = df1.merge(Offense_Y_Removed4_std,how = 'left',  on ='PlayId')
    
# nearest offenders to defender
    dis_to_closest_offender = pd.DataFrame()
    for playid in df3['PlayId'].unique():
        offense = df2.loc[df2['PlayId'] == playid]
        defence = df3.loc[df3['PlayId'] == playid]
        ary = scipy.spatial.distance.cdist(defence[['New_X','New_Y']], offense[['New_X','New_Y']], metric='euclidean')
        ary.sort(axis=1)
        ary = pd.DataFrame(data = ary)
        ary['PlayId'] = playid
        ary.reset_index(drop=True, inplace=True)
        ary = pd.concat([ary, defence[['NflId']].reset_index(drop=True)], axis=1)
        dis_to_closest_offender = dis_to_closest_offender.append(ary)
    df3 = df3.merge(dis_to_closest_offender,how = 'left',on = ['PlayId','NflId'])

# personnel_features
    def defense_formation(l):
        dl = 0
        lb = 0
        db = 0
        other = 0

        for position in l:
            sub_string = position.split(' ')
            if sub_string[1] == 'DL':
                dl += int(sub_string[0])
            elif sub_string[1] in ['LB','OL']:
                lb += int(sub_string[0])
            else:
                db += int(sub_string[0])

        counts = (dl,lb,db,other)

        return counts
    def offense_formation(l):
        qb = 0
        rb = 0
        wr = 0
        te = 0
        ol = 0

        sub_total = 0
        qb_listed = False
        for position in l:
            sub_string = position.split(' ')
            pos = sub_string[1]
            cnt = int(sub_string[0])

            if pos == 'QB':
                qb += cnt
                sub_total += cnt
                qb_listed = True
            # Assuming LB is a line backer lined up as full back
            elif pos in ['RB','LB']:
                rb += cnt
                sub_total += cnt
            # Assuming DB is a defensive back and lined up as WR
            elif pos in ['WR','DB']:
                wr += cnt
                sub_total += cnt
            elif pos == 'TE':
                te += cnt
                sub_total += cnt
            # Assuming DL is a defensive lineman lined up as an additional line man
            else:
                ol += cnt
                sub_total += cnt

        # If not all 11 players were noted at given positions we need to make some assumptions
        # I will assume if a QB is not listed then there was 1 QB on the play
        # If a QB is listed then I'm going to assume the rest of the positions are at OL
        # This might be flawed but it looks like RB, TE and WR are always listed in the personnel
        if sub_total < 11:
            diff = 11 - sub_total
            if not qb_listed:
                qb += 1
                diff -= 1
            ol += diff

        counts = (qb,rb,wr,te,ol)

        return counts
    def split_personnel(s):
        splits = s.split(',')
        for i in range(len(splits)):
            splits[i] = splits[i].strip()

        return splits    
    def personnel_features(df):
        personnel = df[['GameId','PlayId','OffensePersonnel','DefensePersonnel']].drop_duplicates()
        personnel['DefensePersonnel'] = personnel['DefensePersonnel'].apply(lambda x: split_personnel(x))
        personnel['DefensePersonnel'] = personnel['DefensePersonnel'].apply(lambda x: defense_formation(x))
        personnel['num_DL'] = personnel['DefensePersonnel'].apply(lambda x: x[0])
        personnel['num_LB'] = personnel['DefensePersonnel'].apply(lambda x: x[1])
        personnel['num_DB'] = personnel['DefensePersonnel'].apply(lambda x: x[2])

        personnel['OffensePersonnel'] = personnel['OffensePersonnel'].apply(lambda x: split_personnel(x))
        personnel['OffensePersonnel'] = personnel['OffensePersonnel'].apply(lambda x: offense_formation(x))
        personnel['num_QB'] = personnel['OffensePersonnel'].apply(lambda x: x[0])
        personnel['num_RB'] = personnel['OffensePersonnel'].apply(lambda x: x[1])
        personnel['num_WR'] = personnel['OffensePersonnel'].apply(lambda x: x[2])
        personnel['num_TE'] = personnel['OffensePersonnel'].apply(lambda x: x[3])
        personnel['num_OL'] = personnel['OffensePersonnel'].apply(lambda x: x[4])

        # Let's create some features to specify if the OL is covered
        personnel['OL_diff'] = personnel['num_OL'] - personnel['num_DL']
        personnel['OL_TE_diff'] = (personnel['num_OL'] + personnel['num_TE']) - personnel['num_DL']
        # Let's create a feature to specify if the defense is preventing the run
        # Let's just assume 7 or more DL and LB is run prevention
        personnel['run_def'] = (personnel['num_DL'] + personnel['num_LB'] > 6).astype(int)

        personnel.drop(['OffensePersonnel','DefensePersonnel'], axis=1, inplace=True)
        
        return personnel
    
    personnel = personnel_features(df1)   
    df1 = df1.merge(personnel,how = 'left',  on ='PlayId')
    
#select useful columns    
    rusher = df1[['PlayId','TimeDelta','Team','PlayerAge','PlayerHeight','PlayerWeight','New_X','New_Y', \
                 'Orientation_std','Dir_std','Dis_YardLine','Horizontal Speed','Vertical Speed','S','A','Dis','Position',\
                 '# of Handoff Play','Quarter','GameClock','Down','Distance','OffenseFormation',\
                  'DefendersInTheBox','HomeScoreBeforePlay','VisitorScoreBeforePlay',\
                 'Offense_X_Removed2_std','Offense_Y_Removed2_std','Offense_X_Removed4_std','Offense_Y_Removed4_std',\
                 'Defense_X_std','Defense_Y_std','Defense_X_Removed2_std','Offense_Y_Removed2_std','Defense_X_Removed4_std',\
                 'Defense_Y_Removed4_std','num_DL','num_LB','num_DB','num_QB','num_RB','num_WR','num_TE','num_OL','OL_diff','OL_TE_diff',\
                 'run_def']]
    rusher = rusher.sort_values('PlayId')
    game = df1[['PlayId','Cleaned_GameWeather','Humidity','Temperature', \
              'Week']]
    game = game.sort_values('PlayId')
    offender = df2[['PlayId','PlayerAge','PlayerHeight','PlayerWeight','New_X','New_Y','Orientation_std','Dir_std','back_oriented_down_field',\
                  'Dis_YardLine','Horizontal Speed','Vertical Speed','S','A','Dis','Position','Distance_to_Rusher',\
                   'Degree_to_Rusher','Degree_Diff']]
    offender = offender.sort_values(['PlayId','New_Y'])
    defender = df3[['PlayId','PlayerAge','PlayerHeight','PlayerWeight','New_X','New_Y','Orientation_std','Dir_std','back_oriented_down_field',\
                  'Dis_YardLine','Horizontal Speed','Vertical Speed','S','A','Dis','Position','Distance_to_Rusher',\
                   'Degree_to_Rusher','Degree_Diff','Min_Time_Tacke']+list(range(5))]
    defender = defender.sort_values(['PlayId','New_Y'])
    
    y = df1[['PlayId','Yards']].sort_values('PlayId').drop('PlayId',axis = 1)
#get_dummy
    def encoding_cat(df):
        for cat in df.dtypes[df.dtypes=='object'].index.tolist():
            a = pd.get_dummies(df[cat])
            df = df.drop(cat,axis = 1)
            df = pd.concat([df,a],axis = 1)
        return df
    rusher = encoding_cat(rusher)
    game = encoding_cat(game)
    offender = encoding_cat(offender)
    defender = encoding_cat(defender)
    
    return rusher, game, offender, defender, y

In [168]:
rusher, game, offender, defender, y = split_data(train)

C:\Users\yam\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [179]:
rusher = rusher.round(3) 
game = game.round(3) 
offender = offender.round(3) 
defender = defender.round(3) 

In [180]:
offender_players = [offender.drop('PlayId',axis = 1).iloc[np.arange(k, len(offender), 10)].reset_index(drop = True) for k in range(10)]
offender_players = np.hstack([t.values for t in offender_players])
offender_players.shape

(23171, 340)

In [181]:
defender_players = [defender.drop('PlayId',axis = 1).iloc[np.arange(k, len(defender), 11)].reset_index(drop = True) for k in range(11)]
defender_players = np.hstack([t.values for t in defender_players])
defender_players.shape

(23171, 462)

In [182]:
X = np.hstack([rusher.values,game.values,offender_players,defender_players])
Y = np.zeros((y.Yards.shape[0], 199))
for idx, target in enumerate(list(y.Yards)):
    Y[idx][99 + target] = 1

In [189]:
print(np.any(np.isnan(X)))
print(np.any(np.isfinite(X)))

False
True


In [184]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=12345)

In [ ]:
class Metric(Callback):
    def __init__(self, model, callbacks, data):
        super().__init__()
        self.model = model
        self.callbacks = callbacks
        self.data = data

    def on_train_begin(self, logs=None):
        for callback in self.callbacks:
            callback.on_train_begin(logs)

    def on_train_end(self, logs=None):
        for callback in self.callbacks:
            callback.on_train_end(logs)

    def on_epoch_end(self, batch, logs=None):
        X_train, y_train = self.data[0][0], self.data[0][1]
        y_pred = self.model.predict(X_train)
        y_true = np.clip(np.cumsum(y_train, axis=1), 0, 1)
        y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
        tr_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_train.shape[0])
        tr_s = np.round(tr_s, 6)
        logs['tr_CRPS'] = tr_s

        X_valid, y_valid = self.data[1][0], self.data[1][1]

        y_pred = self.model.predict(X_valid)
        y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
        y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
        val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_valid.shape[0])
        val_s = np.round(val_s, 6)
        logs['val_CRPS'] = val_s
        print('tr CRPS', tr_s, 'val CRPS', val_s)

        for callback in self.callbacks:
            callback.on_epoch_end(batch, logs)

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=X.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(199, activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=[])

In [ ]:
es = EarlyStopping(monitor='val_CRPS', 
                   mode='min',
                   restore_best_weights=True, 
                   verbose=2, 
                   patience=5)
es.set_model(model)

In [ ]:
metric = Metric(model, [es], [(X_train,y_train), (X_val,y_val)])

In [ ]:
%%time
model.fit(X_train, y_train, callbacks=[metric], epochs=100, batch_size=1024)